# Start Simulation

In [1]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [2]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [3]:
usd_path = f"omniverse://localhost/Users/{user}/uva_sac.usd"

In [4]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-a9118969-982e-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[3.920s] [ext: omni.kit.menu.create-1.0.2] startup
[3.923s] [ext: omni.mdl-0.1.0] startup
[3.960s] [ext: omni.kit.window.file_importer-1.0.4] startup
[3.961s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[3.963s] [ext: omni.kit.material.library-1.3.10] startup
[3.969s] [ext: omni.kit.window.drop_support-1.0.0] startup
[3.971s] [ext: omni.kit.window.file-1.3.16] startup
[3.975s] [ext: omni.kit.context_menu-1.3.9] startup
[3.982s] [ext: omni.kit.window.property-1.6.3] startup
[3.984s] [ext: omni.kit.window.content_browser-2.4.28] startup
[4.001s] [ext: omni.kit.widget.stage-2.6.15] startup
[4.010s] [ext: omni.isaac.version-1.0.0] startup
[4.011s] [ext: omni.kit.property.usd-3.14.9] startup
[4.066s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[4.072s] [ext: omni.hydra.rtx-0.1.0] startup
[4.117s] [ext: omni.renderer-rtx-0.0.0] startup
[4.122s] [ext: omn

In [5]:
# set log level
import logging
import carb

logging.getLogger("omni.hydra").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.urdf").setLevel(logging.ERROR)
logging.getLogger("omni.physx.plugin").setLevel(logging.ERROR)

logging.getLogger("omni.isaac.synthetic_utils").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.synthetic_utils.syntheticdata").setLevel(logging.ERROR)
logging.getLogger("omni.hydra.scene_delegate.plugin").setLevel(logging.ERROR)


l = carb.logging.LEVEL_ERROR
carb.settings.get_settings().set("/log/level", l)
carb.settings.get_settings().set("/log/fileLogLevel", l)
carb.settings.get_settings().set("/log/outputStreamLevel", l)

# # This logged error is printed as it should
# carb.log_error("ERROR")
# # This warning is printed but should not
# carb.log_warn("WARNING")

# Config

In [6]:
from task.config import DATA_PATH, FEATURE_PATH
task_type = "Table"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

In [7]:
pause

NameError: name 'pause' is not defined

# Init Env

In [8]:
from uva_env import UvaEnv
env = UvaEnv()

In [9]:
from task.utils import add_scene_default
add_scene_default()

In [10]:
print(list(env.stage.TraverseAll()))

[Usd.Prim(</physicsScene>), Usd.Prim(</World>), Usd.Prim(</World/defaultLight>), Usd.Prim(</World/base>), Usd.Prim(</World/base/Apperson_CoffeeTable_Top>), Usd.Prim(</World/base/Apperson_CoffeeTable_Leg>), Usd.Prim(</World/base/Looks>), Usd.Prim(</World/base/Looks/Legs>), Usd.Prim(</World/base/Looks/Legs/Shader>), Usd.Prim(</World/base/Looks/Tabletop>), Usd.Prim(</World/base/Looks/Tabletop/Shader>), Usd.Prim(</World/render>), Usd.Prim(</World/render/camera_main>), Usd.Prim(</World/render/camera_main/OmniverseKitViewportCameraMesh>), Usd.Prim(</World/render/camera_main/OmniverseKitViewportCameraMesh/CameraModel>), Usd.Prim(</World/objects>), Usd.Prim(</OmniverseKit_Persp>), Usd.Prim(</OmniverseKit_Front>), Usd.Prim(</OmniverseKit_Top>), Usd.Prim(</OmniverseKit_Right>), Usd.Prim(</Render>), Usd.Prim(</Render/RenderProduct_Viewport>), Usd.Prim(</Render/Vars>), Usd.Prim(</Render/Vars/LdrColor>)]


In [11]:
env.clean()
env.world.step(render=True)

# Scene

In [12]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [13]:
# add base
scene.add_base_asset()

asset_prim: omniverse://localhost/Users/yizhou/Asset/T/Appleseed/Appleseed_CoffeeTable.usd


In [14]:
# add room
# scene.add_room()

In [15]:
env.world.step(render=True)

# Reward

In [16]:
from uv.reward import Rewarder

In [17]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [18]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras()
render.set_cameras()

xform_mat:  ( (-1, 0, 0, 0), (0, -1, 0, 0), (0, 0, 1, 0), (0, 56, 200, 1) )


In [19]:
pause

NameError: name 'pause' is not defined

#  Learning

In [20]:
# from learning.network.resnet import ResNetFeatureExtractor

# feature extraction
from learning.utils import extract_image_clip_feature_and_save, obtain_action_from_trainer
from transformers import CLIPProcessor, CLIPModel

feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [21]:
# replay buffer
import json
from learning.replay_buffer import ReplayBuffer
from learning.config import *

buffer = ReplayBuffer(max_size=2000)

In [22]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()
target_qf1 = QFunction()
target_qf2 = QFunction()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = False, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

# Trajectory

In [23]:
use_network = True
debug = True

In [24]:
total_traj = 0
total_step = 0

---------------------

In [25]:
# traj config
for traj_id in range(20):
    total_traj += 1
    
    base_asset_id = 0
    env.scene.base_asset_id = base_asset_id
    env.scene.traj_id = traj_id
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))

    # base
    # scene.add_base_asset()
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    
    ## extract feature
    
#     extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#         feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
        f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(5):
        total_step += 1
        
        # sample an object
        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if not use_network or total_traj < 10 or np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            image_feature_file = f"{image_folder}/{step}.pt"
            
            object_info = env.scene.objects[-1]
            object_type = object_info["type"]
            obj_name = object_info["name"][:-4]
            object_feature_file = os.path.join(FEATURE_PATH, object_type, obj_name + ".pt")
            x, y = obtain_action_from_trainer(image_feature_file, object_feature_file, trainer, 
                                              scaler=np.exp(- total_traj / 100))
            object_info["use_network"] = True
        
        # load the object into the scene
        env.put_last_object((x, y)) 
        env.world.step(render=True)
        
        # register the object to the world for physics update
        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
#         extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#             feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))
        extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
            f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)
        
        ## trainer nework
        if use_network and total_step % UPDATE_TRAINER_STEPS == 0 and total_traj > 5:
            batch = buffer.sample_batch(batch_size = BATCH_SIZE)
            trainer.update(batch)
            
            if debug and total_step % 10 == 0:
                rewards = batch['rewards'].to(device)
                terminals = batch['terminals'].to(device)
                obs = batch['observations'].to(device)
                actions = batch['actions'].to(device)
                next_obs = batch['next_observations'].to(device)
                obj_features = batch['object_features'].to(device)
                
                dist = trainer.policy(obs, obj_features)
                pred = trainer.qf1(obs, obj_features, actions)
                print(f"debug {total_traj}/{total_step}", #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
                      "\n pred:", pred.flatten().tolist(),
                      "\n rewards: ", rewards.flatten().tolist())


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    # print("record: ", record)
    
    # Add record to buffer
    buffer.add_scene_sample(record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)

-0.1.0] startup
[13.053s] [ext: omni.isaac.debug_draw-0.1.2] startup
[13.062s] [ext: omni.graph.window.core-1.23.4] startup
[13.074s] [ext: omni.graph.instancing-1.1.4] startup
[13.083s] [ext: omni.kit.menu.edit-1.0.6] startup
[13.087s] [ext: omni.isaac.isaac_sensor-1.0.2] startup
2022-12-24 21:27:52 [13,070ms] [Warning] [omni.physx.plugin] Deprecated: getSimulationEventStream is deprecated, please use getSimulationEventStreamV2
[13.108s] [ext: omni.graph.tutorials-1.1.2] startup
[13.139s] [ext: omni.graph.window.action-1.3.8] startup
[13.142s] [ext: omni.kit.widget.live-0.1.0] startup
[13.147s] [ext: omni.kit.widget.layers-1.5.17] startup
[13.174s] [ext: omni.graph.bundle.action-1.0.0] startup
[13.174s] [ext: omni.isaac.range_sensor-0.4.3] startup
[13.213s] [ext: omni.kit.property.layer-1.1.2] startup
[13.216s] [ext: omni.replicator.isaac-1.3.2] startup
[13.352s] [ext: omni.isaac.kit-0.2.1] startup
[13.353s] [ext: omni.graph.window.generic-1.3.8] startup
[13.355s] [ext: omni.isaac.uti

2022-12-24 21:33:27 [347,105ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2022-12-24 21:33:27 [347,417ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:33:27 [347,417ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2022-12-24 21:33:27 [347,417ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:33:27 [347,417ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2022-12-24 21:33:27 [347,417ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:33:27 [347,417ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2022-12-24 21:33

2022-12-24 21:33:27 [347,950ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/MagazineStack02.usd
2022-12-24 21:33:28 [348,297ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Magazine

2022-12-24 21:33:28 [349,062ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Vase/RecycledGlassVase_Medium.usd
2022-12-24 21:33:29 [349,634ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Vase

2022-12-24 21:33:30 [350,387ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Dinnerware/plate_large.usd
2022-12-24 21:33:30 [350,861ms] [War

2022-12-24 21:33:32 [352,693ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2022-12-24 21:33:32 [353,018ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware_01

2022-12-24 21:33:32 [353,018ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_01

2022-12-24 21:33:32 [353,018ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2022-12-24 21:33:32 [353,018ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware_01

2022-12-24 21:33:32 [353,018ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_01

2022-12-24 21:33:32 [353,018ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase


2022-12-24 21:33:33 [353,593ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/GadgetsAndTool/coffee_grinder.usd
2022-12-24 21:33:33 [354,029ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/GadgetsAndTool

2022-12-24 21:33:34 [354,467ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/MagazineStack02.usd
2022-12-24 21:33:34 [355,000ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Magazine

2022-12-24 21:33:35 [355,703ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2022-12-24 21:33:35 [355,991ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:or

...

2022-12-24 21:35:01 [441,627ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Magazine_01

2022-12-24 21:35:01 [441,787ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2022-12-24 21:35:01 [442,097ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:35:01 [442,097ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_01

2022-12-24 21:35:01 [442,097ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:35:01 [442,097ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_01

2022-12-24 21:35:01 [442,097ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022

2022-12-24 21:35:02 [442,989ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Dinnerware_01

2022-12-24 21:35:03 [443,370ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2022-12-24 21:35:03 [443,682ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware_01

2022-12-24 21:35:03 [443,682ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:35:03 [443,683ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2022-12-24 21:35:03 [443,683ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_01

2022-12-24 21:35:03 [443,683ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinn

2022-12-24 21:35:04 [444,252ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/GadgetsAndTool/red_grinder.usd
2022-12-24 21:35:04 [444,557ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/GadgetsAndTool

2022-12-24 21:35:04 [444,914ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
debug 20/96 
 pred: [0.8869379162788391, 1.1302582025527954, 1.2380108833312988, 1.4648407697677612, 1.58838951587677, 1.1887164115905762, 0.7754154205322266, 1.2283811569213867] 
 rewards:  [-0.4306640625, -0.43062591552734375, -0.43062591552734375, 0.30604949593544006, -2.6915297508239746, -0.43062591552734375, -1.8583145141601562, 0.9903863072395325]
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/GadgetsAndTool/brass_grinder.usd
2022-12-24 21:35:05 [445,586ms] [Warn

2022-12-24 21:35:08 [448,530ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Tchotchke

2022-12-24 21:35:08 [448,813ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2022-12-24 21:35:09 [449,122ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:35:09 [449,122ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:35:09 [449,122ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:35:09 [449,122ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:35:09 [449,122ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Fruit

2022-12-24 21:35:0

debug 20/100 
 pred: [1.1349120140075684, 0.9706529378890991, 0.8648807406425476, 1.0434240102767944, 1.0830384492874146, 1.3424099683761597, 1.1420379877090454, 1.1070476770401] 
 rewards:  [0.9903863072395325, -0.43062591552734375, -0.4306602478027344, -0.43062591552734375, -0.43062591552734375, -0.347426176071167, 0.9928817749023438, -0.4306640625]


# Debug

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

##  Debug Training

In [ ]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(100):
    replay = json.load(open(f"{DATA_PATH}/{task_type}/{side_choice}/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [ ]:
len(buffer.dataset)

In [ ]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()

target_qf1 = QFunction()
target_qf1.eval()

target_qf2 = QFunction()
target_qf2.eval()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = True, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

In [ ]:
self = trainer

In [ ]:
for _ in range(500):
    batch = buffer.sample_batch(batch_size=6)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 10 == 0:
        dist = self.policy(obs, obj_features)
        pred = self.qf1(obs, obj_features, actions)
        print(_, "\n dist   : ", [round(x, 3) for x in dist.mean.flatten().tolist()], 
                  [round(x, 3) for x in dist.stddev.flatten().tolist()],
              "\n pred   :", [round(x, 3) for x in pred.flatten().tolist()],
              "\n rewards: ", [round(x, 3) for x in rewards.flatten().tolist()])
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())

In [ ]:
dist = self.policy(obs.to(device), obj_features.to(device))

In [ ]:
dist.normal_mean, dist.stddev

In [ ]:
dist.sample(scaler = 0.2)

In [ ]:
self = dist

In [ ]:
scaler = 0.1

In [ ]:
self.normal_mean + scaler * self.normal_std * MultivariateDiagonalNormal(
    torch.zeros(self.normal_mean.size()).to(self.device),
    torch.ones(self.normal_std.size()).to(self.device)
).sample()

# distribution

In [ ]:
from learning.distributions import MultivariateDiagonalNormal

In [ ]:
dist = JustNormal(dist.mean, dist.stddev)

In [ ]:
dist.rsample_and_logprob()